# Course Assignment Reporting

## Authors

R. Treharne

## Overview

This notebook will allow a module organiser to generate a spreadsheet containing:

+ A sheet for each SpeedGrader assignment containing all submission info and grades.
+ A summary sheet for each SpeedGrader assignment containing the average score awarded by each unique marker.

## Requirements

If you are running this notebook using Google Colab then use the cell immediately below to install the `cavasapi` Python module.

In [ ]:
# Install canvasapi module if running this in Colab

!pip install canvasapi

## Usage

Run the cells below in order and enter the required information when prompted.

In [ ]:
# Important. Run this cell.
# Nothing will happen, but you will need to run this cell before running the next cell.

# Requirements
from canvasapi import Canvas
import pandas as pd
import getpass
import datetime

def make_clickable(val):
    return f'<a target="_blank" href="{val}">{val}</a>'

def get_assigment_submissions(canvas, course_id, assignment_id):
  course = canvas.get_course(course_id)
  assignment = course.get_assignment(assignment_id)
  submissions = [x for x in assignment.get_submissions(include=["submission_comments"])]
  return submissions

def get_user_by_id(canvas, user_id):
  user = canvas.get_user(user_id)
  return(user)

def get_student_users_by_course(canvas, course_id):
  course = canvas.get_course(course_id)

  student_dict = {}
  enrollments = [x for x in course.get_enrollments() if x.type == "StudentEnrollment"]
  for e in enrollments:
    student_dict[e.user_id] = {
        "login_id": e.user["login_id"],
        "sis_user_id": e.user["sis_user_id"],
        "sortable_name": e.user["sortable_name"]
    }
  return student_dict

def generate_report(canvas, course_id, assignment_id, student_dict, submissions, user_group_map):

  rows = []
  for i, sub in enumerate(submissions):
    try:

      row = {
          "login_id": student_dict[sub.user_id]["login_id"],
          "sis_user_id": student_dict[sub.user_id]["sis_user_id"],
          "sortable_name": student_dict[sub.user_id]["sortable_name"],
          "workflow_state": sub.workflow_state,
          "score": sub.score,
          "seconds_late": sub.seconds_late,
          "academic_advisor": user_group_map[sub.user_id],
          "url": "https://liverpool.instructure.com/courses/{0}/gradebook/speed_grader?assignment_id={1}&student_id={2}".format(course_id, assignment_id, sub.user_id)
      }
      #row["url"] = "https://liverpool.instructure.com/courses/{0}/gradebook/speed_grader?assignment_id={1}&student_id={2}".format(course_id, assignment_id, sub.user_id)
      rows.append(row)

    except:
      continue
  df = pd.DataFrame(rows)
  return df

def create_user_group_map(course):
  user_group_dict = {}
  groups = [x for x in course.get_groups()]
  for group in groups:
    for member in [x for x in group.get_memberships()]:
      user_group_dict[member.user_id] = group.name
  return user_group_dict

def save_file(canvas, fname, assignments_dict):
  with pd.ExcelWriter(fname) as writer:
    for course_id in assignments_dict:
      print("Getting user data for course {0}. This might take a while ...".format(course_id))
      student_dict = get_student_users_by_course(canvas, course_id)
      course = canvas.get_course(course_id)
      user_group_map = create_user_group_map(course)
      assignments = assignments_dict[course_id]["assignments"]
      for assignment in assignments:
        print(course_id, assignment)
        submissions = get_assigment_submissions(canvas, course_id, assignment["id"])
        df = generate_report(canvas, course_id, assignment["id"], student_dict, submissions, user_group_map)
        df = df.sort_values(by=["sortable_name"])


        df.loc[df["workflow_state"]=="submitted", "workflow_state"] = "ungraded"
        df.loc[(df["workflow_state"]=="graded") & (df["score"].isnull()), "workflow_state"] = "EX"


        df.to_excel(writer, sheet_name=assignment["label"], index=False)
        df.style.format({'url': make_clickable})

        summary = pd.DataFrame(df.groupby(by=["academic_advisor"])["score"].mean())
        summary.to_excel(writer, sheet_name="{} - SuperSummary".format(assignment["label"][:11]), index=True)
      print("Done. Saved to {0}".format(fname))


In [ ]:
API_URL = input("Enter your canvas URL (e.g. https://canvas.liverpool.ac.uk)")
API_KEY = getpass.getpass("Enter your Canvas API Key")

# create a canvas object
canvas = Canvas(API_URL, API_KEY)

datestamp = datetime.datetime.now().strftime("%Y%m%dT%H%M%S")
fname = "{0}_assignment_report.xlsx".format(datestamp) # Generate dynamic filename

course_id = input("Input the Canvas course ID (e.g 69023")

print("Getting assignments ...")
assignments = [x for x in canvas.get_course(course_id).get_assignments()]

assignment_filter = input("Enter a filter for the assignment name (e.g. LIFE223.), or leave blank to get all assignments: ")
assignments_list = [{"id": a.id, "label": a.name[:30]} for a in assignments if assignment_filter in a.name and a.workflow_state != "unpublished" ]

# create an input dictionary using course and assignment ids
# {course_id: {"assignments": [{"id": assignment_id, "label": <label>}]}
assignments_dict = {
    course_id: {"assignments": assignments_list},
}

save_file(canvas, fname, assignments_dict)